In [ ]:
!pip install google-genai pandas numpy python-dotenv json5

In [ ]:
# --- 1. Imports und Basis-Konfiguration ---
import os
import json
import warnings
from datetime import datetime, timedelta
from pathlib import Path

import pandas as pd
import numpy as np
from dotenv import load_dotenv
try:
    from google import genai
except ImportError:
    raise ImportError("Installiere google-genai: pip install google-genai")

# Warnungen unterdrücken (für saubere Ausgabe)
warnings.filterwarnings('ignore')

# Lade Umgebungsvariablen (API-Key)
load_dotenv()  # Lädt GEMINI_API_KEY aus .env Datei

# --- 2. Zentral Konfiguration (skalierbar - nur hier anpassen!) ---
CONFIG = {
    # Aktienliste (erweiterbar um beliebige Ticker)
    "AKTIEN_TICKER": ["ASML", "META", "AMZN", "TSLA", "NVDA"],  # NVIDIA = NVDA (Standard-Ticker)
    # Dateipfade (skalierbar - nur Pfade anpassen)
    "NEWS_CSV_PFAD": "news_6m_finnhub_newsapi.csv",
    "KURS_JSON_ORDNER": "Stockcorse_as_json/",  # z.B. Stockcorse_as_json/AMZN_schlusskurse.json
    # Spaltennamen (passend zu deiner Finnhub-News-CSV)
    "NEWS_SPALTEN": {
        "ticker": "ticker",
        "title": "title",
        "url": "url",
        "published_at": "published_at_utc",
        "provider": "provider",
        "channel": "channel",
        "news_source": "news_source",
        "keyword": "keyword"
    },
    # Gemini Konfiguration
    "GEMINI_MODEL": "gemini-2.5-flash",
    "TEMPERATUR": 0.0,  # Keine Variabilität (objektive Ergebnisse)
    # Filter-Schwellen (anpassbar)
    "MIN_KURS_RELEVANZ": 0.5,  # Mindestscore für Kursrelevanz (0-1)
    "MIN_TREND_RELEVANZ": 0.5   # Mindestscore für Trendrelevanz (0-1)
}

# --- 3. Hilfsfunktionen (skalierbar) ---
def init_gemini_client():
    """
    Lazy-Initialisierung des Gemini-Clients (Singleton)
    Skalierbar für verschiedene Modelle
    """
    if getattr(init_gemini_client, "client", None) is None:
        api_key = os.getenv("GEMINI_API_KEY")
        if not api_key:
            raise EnvironmentError(
                "Gemini API-Key fehlt! Setze die Umgebungsvariable GEMINI_API_KEY oder speichere sie in einer .env Datei."
            )
        genai.configure(api_key=api_key)
        init_gemini_client.client = genai.GenerativeModel(CONFIG["GEMINI_MODEL"])
    return init_gemini_client.client

def lade_news_daten():
    """
    Lädt die News-Daten aus der CSV-Datei (skalierbar für beliebige Ticker)
    Returns: DataFrame mit bereinigten News-Daten
    """
    if not Path(CONFIG["NEWS_CSV_PFAD"]).exists():
        raise FileNotFoundError(f"News-CSV nicht gefunden: {CONFIG['NEWS_CSV_PFAD']}")

    df = pd.read_csv(CONFIG["NEWS_CSV_PFAD"])

    # Prüfe auf erforderliche Spalten
    required_cols = list(CONFIG["NEWS_SPALTEN"].values())
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise KeyError(f"Fehlende Spalten in News-CSV: {missing_cols}")

    # Filter nur relevante Ticker (skalierbar)
    df = df[df[CONFIG["NEWS_SPALTEN"]["ticker"]].isin(CONFIG["AKTIEN_TICKER"])].reset_index(drop=True)

    # Datum formatieren (UTC -> datetime)
    df["published_at_datetime"] = pd.to_datetime(df[CONFIG["NEWS_SPALTEN"]["published_at"]], errors="coerce")

    print(f"✅ {len(df)} News-Einträge geladen (nur für {CONFIG['AKTIEN_TICKER']})")
    return df

def lade_kursdaten_fuer_ticker(ticker):
    """
    Lädt die JSON-Kursdaten für einen einzelnen Ticker (skalierbar)
    Args:
        ticker: Aktien-Ticker (z.B. "AMZN")
    Returns: DataFrame mit Datum und Schlusskurs
    """
    json_pfad = Path(CONFIG["KURS_JSON_ORDNER"]) / f"{ticker}_schlusskurse.json"
    if not json_pfad.exists():
        raise FileNotFoundError(f"Kurs-JSON nicht gefunden: {json_pfad}")

    # JSON laden und parsen
    with open(json_pfad, "r", encoding="utf-8") as f:
        kurs_daten = json.load(f)

    # JSON-Struktur aufbereiten: [{"('Schlusskurs', 'AMZN')": 206.16}, ...] -> DataFrame
    schlusskurse = []
    for entry in kurs_daten:
        for key, value in entry.items():
            # Extrahiere Wert (ignoriere Key-String, nur Wert relevant)
            schlusskurse.append(value)

    # Datum generieren (letzte 6 Monate, passend zur Anzahl der Kurseinträge)
    anzahl_tage = len(schlusskurse)
    start_datum = datetime.now() - timedelta(days=anzahl_tage)
    daten = pd.date_range(start=start_datum, periods=anzahl_tage, freq="D")

    df_kurs = pd.DataFrame({
        "Datum": daten,
        "Schlusskurs": schlusskurse,
        "Ticker": ticker
    })

    return df_kurs

def lade_alle_kursdaten():
    """
    Lädt Kursdaten für alle Ticker in CONFIG (skalierbar)
    Returns: Zusammengeführtes DataFrame mit Kursdaten aller Aktien
    """
    alle_kursdaten = []
    for ticker in CONFIG["AKTIEN_TICKER"]:
        try:
            df_ticker = lade_kursdaten_fuer_ticker(ticker)
            alle_kursdaten.append(df_ticker)
        except FileNotFoundError as e:
            print(f"⚠️ {e} - Überspringe diesen Ticker")

    df_kurs_gesamt = pd.concat(alle_kursdaten, ignore_index=True)
    print(f"✅ Kursdaten für {len(alle_kursdaten)} Ticker geladen")
    return df_kurs_gesamt

# --- 4. Filterfunktionen (Kurs- und Trendrelevanz) ---
def bewerte_relevanz(news_text, ticker):
    """
    Bewertet eine News nach:
    1. Kursrelevanz (0-1): Potenzial zur Beeinflussung des Aktienkurses
    2. Trendrelevanz (0-1): Potenzial zur Beeinflussung des Trends (up/down)
    3. Trendrichtung (up/down/neutral)
    Skalierbar: Anpassbare Prompt-Regeln
    """
    client = init_gemini_client()

    prompt = f"""
    Du bist ein erfahrener Finanzanalyst. Bewerte die folgende News für den Aktien-Ticker {ticker}:

    ANWEISUNGEN:
    1. BEWERTE KURSRELEVANZ (0-1):
       - 0: Kein Einfluss auf den Aktienkurs (z.B. unwichtige Nachrichten)
       - 1: Starker Einfluss (z.B. Produktionsbruch, Umsatzmeldungen, große Investitionen)
    2. BEWERTE TRENDELEVANZ (0-1):
       - 0: Kein Einfluss auf den Trend (up/down)
       - 1: Starker Einfluss auf den Trend (z.B. Produktionsbruch → down-Trend, hohe Umsätze → up-Trend)
    3. GIB TRENDRICHTUNG AN:
       - up: News begünstigt steigenden Kurs
       - down: News begünstigt fallenden Kurs
       - neutral: Keine klare Richtung

    GIB DIE ANTWORT IN FOLGENDEM FORMAT ZURÜCK (NUR JSON, KEINE WEITEREN TEXTEN!):
    {{"kurs_relevanz": 0.8, "trend_relevanz": 0.7, "trend_richtung": "down"}}

    NEWS-TEXT: {news_text[:3000]}  # Begrenze Länge für Performance
    """

    try:
        response = client.generate_content(prompt)
        # JSON parsen
        relevanz_ergebnis = json.loads(response.text.strip())
        return (
            relevanz_ergebnis["kurs_relevanz"],
            relevanz_ergebnis["trend_relevanz"],
            relevanz_ergebnis["trend_richtung"]
        )
    except Exception as e:
        print(f"⚠️ Fehler bei Relevanzbewertung: {e}")
        return 0.0, 0.0, "neutral"

def filter_news_nach_relevanz(df_news):
    """
    Filtert News nach Kurs- und Trendrelevanz (skalierbar via CONFIG-Schwellen)
    Returns: DataFrame mit gefilterten News + Relevanz-Scores
    """
    df_copy = df_news.copy()

    # Initialisiere Spalten für Relevanz
    df_copy["kurs_relevanz"] = 0.0
    df_copy["trend_relevanz"] = 0.0
    df_copy["trend_richtung"] = "neutral"

    print(f"🔍 Bewerte Relevanz von {len(df_copy)} News-Einträgen...")

    # Iteriere über alle News (skalierbar für große Datensätze)
    for idx, row in df_copy.iterrows():
        ticker = row[CONFIG["NEWS_SPALTEN"]["ticker"]]
        # Kombiniere Title + Keyword für bessere Bewertung
        news_text = f"{row[CONFIG['NEWS_SPALTEN']['title']]} {row[CONFIG['NEWS_SPALTEN']['keyword']]}"

        # Bewerte Relevanz
        kurs_rel, trend_rel, trend_rich = bewerte_relevanz(news_text, ticker)

        # Speichere Ergebnisse
        df_copy.at[idx, "kurs_relevanz"] = kurs_rel
        df_copy.at[idx, "trend_relevanz"] = trend_rel
        df_copy.at[idx, "trend_richtung"] = trend_rich

    # Filter nach Mindestschwellen (skalierbar via CONFIG)
    df_gefiltert = df_copy[
        (df_copy["kurs_relevanz"] >= CONFIG["MIN_KURS_RELEVANZ"]) &
        (df_copy["trend_relevanz"] >= CONFIG["MIN_TREND_RELEVANZ"])
    ].reset_index(drop=True)

    print(f"✅ {len(df_gefiltert)} relevante News (von {len(df_copy)} total)")
    return df_gefiltert

# --- 5. Neutralisierung der News (optimiert für Finanztexte) ---
def neutralisiere_news_text(news_text, ticker):
    """
    Neutralisiert News-Text (entfernt Bias/emotionale Sprache)
    Skalierbar für beliebige Ticker
    """
    if not isinstance(news_text, str) or len(news_text) < 10:
        return news_text

    client = init_gemini_client()
    system_prompt = f"""
    Du bist ein neutraler Finanzjournalist. Rewrite den folgenden Text über den Aktien-Ticker {ticker}:
    - Entferne alle emotionalen Ausdrücke (z.B. "katastrophal", "erstaunlich", "fantastisch")
    - Entferne subjektive Meinungen (z.B. "Experten glauben, dass...")
    - Behalte alle objektiven Fakten bei (Produktionsbrüche, Umsatzzahlen, Investitionen etc.)
    - Halte dich strikt an den Originalinhalt (keine Erweiterungen/Reduzierungen)
    - Nutze präzise, faktenbasierte Sprache (typisch für Finanzberichte)
    """

    try:
        response = client.generate_content(
            contents=news_text[:3000],
            generation_config=genai.types.GenerationConfig(
                temperature=CONFIG["TEMPERATUR"],
                max_output_tokens=1500
            ),
            system_instruction=system_prompt
        )
        return response.text
    except Exception as e:
        print(f"⚠️ Fehler bei Neutralisierung: {e}")
        return news_text

def neutralisiere_alle_news(df_gefiltert):
    """
    Neutralisiert alle gefilterten News (skalierbar)
    """
    df_copy = df_gefiltert.copy()
    df_copy["neutraler_text"] = df_copy.apply(
        lambda row: neutralisiere_news_text(
            f"{row[CONFIG['NEWS_SPALTEN']['title']]} {row[CONFIG['NEWS_SPALTEN']['keyword']]}",
            row[CONFIG["NEWS_SPALTEN"]["ticker"]]
        ),
        axis=1
    )
    print("✅ Alle relevanten News neutralisiert")
    return df_copy

# --- 6. Zusammenfassung pro Ticker (skalierbar) ---
def zusammenfasse_news_pro_ticker(df_verarbeitet):
    """
    Erstellt strukturierte Zusammenfassungen pro Ticker (skalierbar)
    Returns: Dictionary mit Zusammenfassungen
    """
    client = init_gemini_client()
    zusammenfassungen = {}

    for ticker in CONFIG["AKTIEN_TICKER"]:
        df_ticker = df_verarbeitet[df_verarbeitet[CONFIG["NEWS_SPALTEN"]["ticker"]] == ticker]

        if len(df_ticker) == 0:
            zusammenfassungen[ticker] = "Keine relevanten News gefunden."
            continue

        # Kombiniere neutralisierte Texte
        combined_text = "\n---\n".join(df_ticker["neutraler_text"].tolist())[:5000]

        prompt = f"""
        Erstelle eine strukturierte Zusammenfassung der relevanten News für {ticker} (letzte 6 Monate):

        STRUKTUR:
        1. Kursrelevante Ereignisse (z.B. Produktionsbrüche, Umsatzmeldungen, Investitionen)
        2. Trendauswirkungen (Welche Richtung (up/down/neutral) wird beeinflusst? Warum?)
        3. Wichtige Fakten (keine Meinungen, nur objekte Informationen)

        ANWEISUNGEN:
        - Maximal 300 Wörter pro Abschnitt
        - Nur Fakten aus den bereitgestellten Texten
        - Neutral und faktenbasiert

        NEWS-TEXTE:
        {combined_text}
        """

        try:
            response = client.generate_content(prompt)
            zusammenfassungen[ticker] = response.text
            print(f"✅ Zusammenfassung für {ticker} erstellt")
        except Exception as e:
            zusammenfassungen[ticker] = f"Fehler bei Zusammenfassung: {str(e)}"

    return zusammenfassungen

# --- 7. Speicherfunktionen (CSV + JSON, skalierbar) ---
def speichere_ergebnisse(df_verarbeitet, zusammenfassungen):
    """
    Speichert die verarbeiteten Daten als CSV und JSON (skalierbar)
    """
    # Speichere verarbeitete News als CSV
    csv_pfad = "verarbeitete_aktien_news.csv"
    df_verarbeitet.to_csv(csv_pfad, index=False, encoding="utf-8")
    print(f"📄 Verarbeitete News gespeichert: {csv_pfad}")

    # Speichere Zusammenfassungen als JSON
    json_pfad = "aktien_news_zusammenfassungen.json"
    with open(json_pfad, "w", encoding="utf-8") as f:
        json.dump(zusammenfassungen, f, indent=4, ensure_ascii=False)
    print(f"📄 Zusammenfassungen gespeichert: {json_pfad}")

    # Optional: Speichere Kursdaten als CSV (für Nachverarbeitung)
    try:
        df_kurs = lade_alle_kursdaten()
        kurs_csv_pfad = "alle_aktien_kursdaten.csv"
        df_kurs.to_csv(kurs_csv_pfad, index=False, encoding="utf-8")
        print(f"📄 Kursdaten gespeichert: {kurs_csv_pfad}")
    except Exception as e:
        print(f"⚠️ Kursdaten konnten nicht gespeichert werden: {e}")

# --- 8. Hauptpipeline (vollständig & skalierbar) ---
def hauptverarbeitung():
    """
    Vollständige Pipeline:
    1. Daten laden → 2. Relevanzfilter → 3. Neutralisierung → 4. Zusammenfassung → 5. Speichern
    """
    print("🚀 Starte Hauptverarbeitung...")

    # Schritt 1: Daten laden
    df_news = lade_news_daten()
    _ = lade_alle_kursdaten()  # Nur zur Prüfung, ob Kursdaten existieren

    # Schritt 2: Filter nach Relevanz (Kurs + Trend)
    df_gefiltert = filter_news_nach_relevanz(df_news)

    if len(df_gefiltert) == 0:
        print("⚠️ Keine relevanten News gefunden!")
        return

    # Schritt 3: Neutralisierung
    df_verarbeitet = neutralisiere_alle_news(df_gefiltert)

    # Schritt 4: Zusammenfassung pro Ticker
    zusammenfassungen = zusammenfasse_news_pro_ticker(df_verarbeitet)

    # Schritt 5: Ergebnisse speichern
    speichere_ergebnisse(df_verarbeitet, zusammenfassungen)

    # Ausgabe der Zusammenfassungen
    print("\n" + "="*50)
    print("📝 FINALE ZUSAMMENFASSUNGEN PRO TICKER")
    print("="*50)
    for ticker, summary in zusammenfassungen.items():
        print(f"\n--- {ticker} ---")
        print(summary)

    return df_verarbeitet, zusammenfassungen

# --- 9. Ausführung der Pipeline ---
if __name__ == "__main__":
    # Starte die vollständige Verarbeitung
    df_verarbeitet, zusammenfassungen = hauptverarbeitung()